In [4]:
import os
import requests
import zipfile
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# Configurações
BASE_URL = "https://dados.cvm.gov.br/dados/"
DEST_DIR = r"E:/Download/cvm"

# Conjunto para rastrear o que já foi mapeado e evitar loops
visited_urls = set()

def is_valid_url(url):
    """Verifica se a URL pertence ao portal de dados e não é um link de saída."""
    parsed = urlparse(url)
    # Garante que só navegamos dentro da pasta /dados/ do servidor da CVM
    return parsed.netloc == "dados.cvm.gov.br" and parsed.path.startswith("/dados/")

def get_content(url):
    """Extrai pastas e arquivos de uma página, filtrando lixo e links circulares."""
    try:
        if url in visited_urls or not is_valid_url(url):
            return [], []
        
        visited_urls.add(url)
        response = requests.get(url, timeout=15)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        links = [node.get('href') for node in soup.find_all('a') if node.get('href')]
        
        pastas = []
        arquivos = []
        
        for link in links:
            # Ignora links de sistema, navegação de 'voltar' e parâmetros de busca
            if link in ['../', './', '/'] or link.startswith('?') or link.startswith('http'):
                # Só processa links absolutos se forem do próprio domínio (via urljoin)
                if not link.startswith('http'):
                    pass 
                else:
                    if not is_valid_url(link): continue

            full_url = urljoin(url, link)
            
            # Critério de separação: se termina com / é pasta, senão checa extensão
            if link.endswith('/'):
                if full_url not in visited_urls:
                    pastas.append(full_url)
            elif link.lower().endswith(('.zip', '.csv')):
                arquivos.append(full_url)
                
        return pastas, arquivos
    except Exception as e:
        print(f"Erro ao acessar {url}: {e}")
        return [], []

def download_and_unzip(url):
    """Lógica de download, extração e limpeza."""
    # Gera o caminho local baseado na estrutura da URL
    relative_path = urlparse(url).path.replace("/dados/", "").lstrip("/")
    # O diretório é tudo exceto o nome do arquivo final
    sub_folders = os.path.split(relative_path)[0]
    local_dir = os.path.join(DEST_DIR, sub_folders)
    
    os.makedirs(local_dir, exist_ok=True)
    file_name = url.split('/')[-1]
    file_path = os.path.join(local_dir, file_name)
    
    # Se o arquivo descompactado (csv) já existe, pula para poupar banda
    if os.path.exists(file_path.replace('.zip', '.csv')):
        return

    print(f"Baixando: {relative_path}")
    try:
        r = requests.get(url, stream=True)
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024*1024): # 1MB chunks
                f.write(chunk)
        
        if file_name.lower().endswith('.zip'):
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(local_dir)
            os.remove(file_path)
    except Exception as e:
        print(f"Falha em {file_name}: {e}")

def run_crawler(start_url):
    """Percorre a árvore de diretórios sem usar recursividade profunda (usando uma pilha)."""
    stack = [start_url]
    
    while stack:
        current_url = stack.pop()
        sub_pastas, arquivos = get_content(current_url)
        
        # 1. Baixa os arquivos encontrados na pasta atual
        for arq in arquivos:
            download_and_unzip(arq)
            
        # 2. Adiciona as novas pastas na pilha para explorar depois
        for pasta in sub_pastas:
            stack.append(pasta)

print(f"Iniciando Crawling em {BASE_URL}...")
run_crawler(BASE_URL)
print("\nFinalizado!")

Iniciando Crawling em https://dados.cvm.gov.br/dados/...
Baixando: SECURIT/DOC/INF_MENSAL_OTS/META/meta_inf_mensal_ots.zip
Baixando: SECURIT/DOC/INF_MENSAL_OTS/DADOS/inf_mensal_ots_2023.zip
Baixando: SECURIT/DOC/INF_MENSAL_OTS/DADOS/inf_mensal_ots_2024.zip
Baixando: SECURIT/DOC/INF_MENSAL_OTS/DADOS/inf_mensal_ots_2025.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/META/meta_inf_mensal_cri.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2019.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2020.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2021.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2022.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2023.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2024.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRI/DADOS/inf_mensal_cri_2025.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRA/META/meta_inf_mensal_cra.zip
Baixando: SECURIT/DOC/INF_MENSAL_CRA/DADOS/inf_mensal_cra_2019.zip
Baixando